In [1]:
from flask import Flask, request, jsonify, url_for, render_template
import uuid
import os
from tensorflow.keras.models import load_model
import numpy as np
from werkzeug.utils import secure_filename
from tensorflow.keras.applications import MobileNet
from PIL import Image, ImageFile
from  io import BytesIO
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.applications.mobilenet import decode_predictions


In [2]:
ALLOWED_EXTENSION = set(['txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'])
IMAGE_HEIGHT = 224 #dimensions
IMAGE_WIDTH = 224 #image dimensions
IMAGE_CHANNELS = 3 #rgb

In [3]:
def allowed_file(filename):
    #check if the filetype is allowed (image file)
    return '.' in filename and filename.rsplit('.', 1)[1] in ALLOWED_EXTENSION

In [4]:
import os
os.getcwd()

'/home/dragos/Downloads/Data'

In [ ]:
app = Flask(__name__)
model = MobileNet(weights='imagenet', include_top=True)

@app.route('/index')
def index():
    return render_template('ImageML.html')

@app.route('/api/image', methods=['POST'])
def upload_image():
    if 'image' not in request.files:
        return render_template('ImageML.html', prediction='No posted image. Should be attribute named image')
    file = request.files['image']
    
    if file.filename =='':
        return render_template('ImageML.html', prediction = 'You did not select an image')
    
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        print("***"+filename)
        x = []
        ImageFile.LOAD_TRUNCATED_IMAGES = False
        img = Image.open(BytesIO(file.read()))
        img.load()
        img  = img.resize((IMAGE_WIDTH, IMAGE_HEIGHT), Image.ANTIALIAS)
        x  = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x  = preprocess_input(x)
        pred = model.predict(x)
        lst =  decode_predictions(pred, top=3)
        
        items = []
        for item in lst[0]:
            items.append({'name': item[1], 'prob': float(item[2])})
        
        response = {'pred': items}
        return render_template('ImageML.html', prediction = 'I would say the image is most likely {}'.format(response))
    else:
        return render_template('ImageML.html', prediction = 'Invalid File extension')

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Apr/2021 14:20:59] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [25/Apr/2021 14:21:47] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [25/Apr/2021 14:22:01] "GET /index HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2021 14:22:02] "GET /static/css/style.css HTTP/1.1" 200 -


***td.jpeg
40960/35363 [==================================] - 0s 0us/step


127.0.0.1 - - [25/Apr/2021 14:25:09] "POST /api/image HTTP/1.1" 200 -


In [13]:
model = MobileNet()

AttributeError: 'str' object has no attribute 'decode'